# Blue Brain BioExplorer
![](../bioexplorer_banner.png)

## Script to create a visualization of the Spike

In [ ]:
from bioexplorer import BioExplorer, Protein, Sugar, Quaternion, MovieMaker
import os

be = BioExplorer()
core = be.core_api()
print(be.version())
status = be.start_model_loading_transaction()

In [ ]:
be.reset_scene()
surface = False
open_spike = False

In [ ]:
'''Resources'''
resource_folder = os.path.abspath('../../tests/test_files')
pdb_folder = os.path.join(resource_folder, 'pdb')
complex_folder = os.path.join(pdb_folder, 'glycans', 'complex')
colormap_folder = os.path.join(resource_folder, 'colormap')

protein_representation = be.protein_representation.ATOMS_AND_STICKS
protein_radius_multiplier = 1.0
glycan_representation = be.protein_representation.ATOMS_AND_STICKS
glycan_radius_multiplier = 1.0

if surface:
    protein_radius_multiplier = 1.0
    protein_representation = BioExplorer.REPRESENTATION_SURFACE
    glycan_representation = be.protein_representation.ATOMS_AND_STICKS

In [ ]:
'''Default side view'''
status = core.set_camera(
    orientation=[0.0, 0.0, -1.0, 0.0],
    position=[0.019, 0.295, 28.535],
    target=[0.019, 0.295, 1.440]
)

In [ ]:
'''Protein'''
name = be.NAME_PROTEIN_S_CLOSED

source = os.path.join(pdb_folder, 'sars-cov-2-v1.pdb')
if open_spike:
    source = os.path.join(pdb_folder, '6vyb.pdb')
    
spike = Protein(
    name=name,
    source=source,
    rotation=Quaternion(0.707, -0.707, 0.0, 0.0)
)

if open_spike:
    name = be.NAME_PROTEIN_S_OPEN
    
status = be.add_protein(
    protein=spike,
    atom_radius_multiplier=protein_radius_multiplier,
    representation=protein_representation,
)
be.apply_default_color_scheme(be.shading_mode.BASIC)

## Add glycans to protein

In [ ]:
glycan_folder = os.path.join(pdb_folder, 'glycans')
complex_folder = os.path.join(glycan_folder, 'complex')
high_mannose_folder = os.path.join(glycan_folder, 'high-mannose')
hybrid_folder = os.path.join(glycan_folder, 'hybrid')
o_glycan_folder = os.path.join(glycan_folder, 'o-glycan')

complex_paths = [
    os.path.join(complex_folder, '33.pdb'),
    os.path.join(complex_folder, '34.pdb'),
    os.path.join(complex_folder, '35.pdb'),
    os.path.join(complex_folder, '36.pdb')
]
high_mannose_paths = [
    os.path.join(high_mannose_folder, '1.pdb'),
    os.path.join(high_mannose_folder, '2.pdb'),
    os.path.join(high_mannose_folder, '3.pdb'),
    os.path.join(high_mannose_folder, '4.pdb')
]
hybrid_paths = [os.path.join(hybrid_folder, '24.pdb')]
o_glycan_paths = [os.path.join(o_glycan_folder, '12.pdb')]

### High mannose

In [ ]:
indices = [61, 122, 234, 603, 709, 717, 801, 1074]

high_mannose_glycans = Sugar(
    rotation=Quaternion(0.707, 0.0, 0.0, 0.707),
    assembly_name=name, name=be.NAME_GLYCAN_HIGH_MANNOSE, 
    protein_name=name, source=high_mannose_paths[0], 
    site_indices=indices,
    representation=glycan_representation,
    atom_radius_multiplier=glycan_radius_multiplier
)
status = be.add_glycan(high_mannose_glycans)
be.apply_default_color_scheme(be.shading_mode.BASIC)

### O-Glycans

In [ ]:
protein_name = be.NAME_PROTEIN_S_CLOSED
if open_spike:
    protein_name = be.NAME_PROTEIN_S_OPEN
    
for index in [323, 325]:
    o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index)
    o_glycan = Sugar(
        assembly_name=name, name=o_glycan_name, source=o_glycan_paths[0],
        protein_name=protein_name, site_indices=[index],
        representation=glycan_representation)
    be.add_sugar(o_glycan)
be.apply_default_color_scheme(be.shading_mode.BASIC)

### Complex

In [ ]:
indices = [17, 74, 149, 165, 282, 331, 343, 616, 657, 1098, 1134, 1158, 1173, 1194]
if open_spike:
    indices = [17, 74, 149, 165, 282, 331, 343, 657, 1098, 1134, 1158, 1173, 1194]

complex_glycans = Sugar(
    rotation=Quaternion(0.0, 0.0, 0.0, 1.0),
    assembly_name=name, name=be.NAME_GLYCAN_COMPLEX, 
    protein_name=name, source=complex_paths[0], 
    site_indices=indices,
    representation=glycan_representation,
    atom_radius_multiplier=glycan_radius_multiplier
)
status = be.add_glycan(complex_glycans)
be.apply_default_color_scheme(be.shading_mode.BASIC)

## Rendering settings

In [ ]:
status = be.set_rendering_quality(be.rendering_quality.HIGH)

## Snaphots

In [ ]:
output_folder = '/tmp'
image_size = [1024, 1024]
image_spp = 64

In [ ]:
def generate_snapshot(top_view, keywords):

    '''Define path'''
    base_name = 'protein_s'
    if open_spike:
        base_name += '_open'
    else:
        base_name += '_closed'
    if surface:
        base_name += '_surface'
    if top_view:
        base_name += '_top'
    else:
        base_name += '_side'
    for keyword in keywords:
        base_name += '_' + keyword
    print('Exporting to ' + output_folder + '/' + base_name + '.png')
        
    '''Camera'''
    if top_view:
        status = core.set_camera(
            orientation=[0.5, -0.5, 0.5, 0.5],
            position=[0.360, -26.702, 0.346],
            target=[0.360, 0.172, 0.346]    
        )
    else:
        if open_spike:
            status = core.set_camera(
                orientation=[1.0, 0.0, 0.0, 0.0],
                position=[-1.117, -0.561, -25.184],
                target=[-1.117, -0.561, 0.620]    
            )
        else:
            status = core.set_camera(
                orientation=[0.707, 0.0, -0.707, 0.0],
                position=[-26.348, 0.172, 0.620],
                target=[-2.728, 0.172, 0.620]
            )
    
    '''Snapshot'''
    mm = MovieMaker(be)
    mm.create_snapshot(
        renderer='advanced',
        path=output_folder, base_name=base_name,
        size=image_size, samples_per_pixel=image_spp)

### Functional regions

In [ ]:
def apply_functional_regions_color_scheme():
    '''RGB color palette for amino acid indices'''
    l = 0.2
    h = 1.0
    grey = [l,l,l]
    dark_green = [0.0, l, 0.0]
    light_green = [0.0, h, 0.0]
    red = [h, 0.0, 0.0]
    green = [0.0, h, 0.0]
    blue = [0.0, 0.0, h]
    orange = [h, h/2.0, 0.0]
    cyan = [h, 0.0, h]

    region_indices_and_colors = [
        [   1, grey ], [   16, blue], [  306, grey], [  330, green], [438, dark_green], 
        [ 507, green], [  522, grey], [  816, red ], [  835, grey ], [908, orange],
        [ 986, grey ], [ 1076, cyan], [ 1274, grey], [ 2000, grey ]
    ]

    '''Build a palette according to region colors'''
    palette = list()
    for index in range(len(region_indices_and_colors)-1):
        for i in range(region_indices_and_colors[index + 1][0] - 
                       region_indices_and_colors[index][0]):
            palette.append(region_indices_and_colors[index][1])

    '''Apply palette to other chains'''
    status = be.set_protein_color_scheme(
        assembly_name=name, name=name, 
        color_scheme=be.protein_color_scheme.CHAINS,
        palette_name='Greys_r', palette_size=5)

    '''Apply palette to region color scheme (optional: only for chain #2)'''
    status = be.set_protein_color_scheme(
        assembly_name=name, name=name, 
        color_scheme=be.protein_color_scheme.REGION, palette=palette,
    #     chain_ids=[2]
    )
    
apply_functional_regions_color_scheme()

In [ ]:
generate_snapshot(False, ['glycans', 'with_regions'])

In [ ]:
generate_snapshot(True, ['glycans', 'with_regions'])

### No functional regions

In [ ]:
status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.protein_color_scheme.CHAINS,
    palette_name='binary_r', palette_size=10)

In [ ]:
generate_snapshot(False, ['glycans'])

In [ ]:
generate_snapshot(True, ['glycans'])

## Brazilian variant

### Modify amino acid sequence

In [ ]:
modified_aa_index = 20
status = be.set_protein_amino_acid(
    assembly_name=name, name=name,
    index=modified_aa_index - 1,
    amino_acid_short_name='ASN'
)

indices = [modified_aa_index]
glycan_variant_name = be.NAME_GLYCAN_COMPLEX + '_variant'
complex_glycans = Sugar(
    assembly_name=name, name=glycan_variant_name,
    protein_name=name, source=complex_paths[0], 
    site_indices=indices,
    representation=glycan_representation,
    atom_radius_multiplier=glycan_radius_multiplier
)
status = be.add_glycan(complex_glycans)

In [ ]:
for model in core.scene.models:
    model_id = model['id']
    model_name = model['name']
    if model_name == glycan_variant_name:
        material_ids = list(be.get_material_ids(model_id)['ids'])
        nb_materials = len(material_ids)
        print(model_name)
        palette = list()
        for i in range(nb_materials):
            palette.append([1, 0.2, 0.2])
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids,
            diffuse_colors=palette, specular_colors=palette
        )        

### Snapshots

In [ ]:
apply_functional_regions_color_scheme()

In [ ]:
generate_snapshot(False, ['glycans', 'with_regions', 'brazilian_variant_red'])

In [ ]:
generate_snapshot(True, ['glycans', 'with_regions', 'brazilian_variant_red'])

### Restore initial amino acid sequence

In [ ]:
modified_aa_index = 20
status = be.set_protein_amino_acid(
    assembly_name=name, name=name,
    index=modified_aa_index - 1,
    amino_acid_short_name='THR'
)

status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.protein_color_scheme.GLYCOSYLATION_SITE,
    palette_name='Set1', palette_size=2)

## Stylish

In [ ]:
be.apply_default_color_scheme(
    shading_mode=be.shading_mode.PERLIN,
    user_parameter=0.03, specular_exponent=5.0, glossiness=0.1)

In [ ]:
be.apply_default_color_scheme(
    shading_mode=be.shading_mode.ELECTRON,
    user_parameter=1.0, glossiness=1.0)

In [ ]:
be.apply_default_color_scheme(
    shading_mode=be.shading_mode.GOODSELL,
    user_parameter=3.0, glossiness=1.0)

## Glycosylation sites

### Sites 20, 331 and 343

In [ ]:
ranges = [20, 20, 331, 331, 343, 343]
status = be.set_protein_amino_acid_sequence_as_ranges(
    assembly_name=name, name=name, amino_acid_ranges=ranges)

status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.protein_color_scheme.AMINO_ACID_SEQUENCE,
    palette_name='Set1', palette_size=2)

### Sites 165 and 234

In [ ]:
ranges = [165, 165, 234, 234]
status = be.set_protein_amino_acid_sequence_as_ranges(
    assembly_name=name, name=name, amino_acid_ranges=ranges)

status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.protein_color_scheme.AMINO_ACID_SEQUENCE,
    palette_name='Set1', palette_size=2)

## Visualization of electromagnetic fields

In [ ]:
be.go_magnetic(colormap_filename=os.path.join(colormap_folder, 'spike_v1.1dt'))

### Snapshots

In [ ]:
status = core.set_camera(
    orientation=[1.0, 0.0, 0.0, 0.0],
    position=[-4.252, -3.626, -24.492],
    target=[-4.252, -3.626, -0.951]
)

In [ ]:
mm = MovieMaker(be)
mm.create_snapshot(
    renderer='advanced',
    size=[512, 512], samples_per_pixel=64,
    path='/tmp', base_name='spike_slice')